In [ ]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import random
from collections import deque

class DataPreprocessor:
    def __init__(self, dataset_path, m=3):
        self.dataset_path = dataset_path
        self.m = m
        self.scaler = StandardScaler()

    def load_and_preprocess_data(self):
        df = pd.read_csv(self.dataset_path, usecols=['_source_source_ip', '_source_destination_ip', 
                                                      '_source_network_bytes', '_source_@timestamp', 'label'])
        df['timestamp'] = pd.to_datetime(df['_source_@timestamp'])
        df['window_id'] = df['timestamp'].dt.floor('T').astype('int64') // 10**9 // 60
        grouped = df.groupby('window_id')
        window_ids = np.array(sorted(grouped.groups.keys()))

        X, y = [], []
        for wid in window_ids:
            window_df = grouped.get_group(wid)
            all_ips = np.unique(np.concatenate([window_df['_source_source_ip'].unique(), window_df['_source_destination_ip'].unique()]))
            ip_to_idx = {ip: i for i, ip in enumerate(all_ips)}
            num_nodes = len(all_ips)
            node_features = np.zeros((num_nodes, 4))
            for ip in all_ips:
                src_bytes = window_df[window_df['_source_source_ip'] == ip]['_source_network_bytes'].sum()
                dst_bytes = window_df[window_df['_source_destination_ip'] == ip]['_source_network_bytes'].sum()
                num_src_connections = len(window_df[window_df['_source_source_ip'] == ip])
                num_dst_connections = len(window_df[window_df['_source_destination_ip'] == ip])
                node_features[ip_to_idx[ip]] = [src_bytes, dst_bytes, num_src_connections, num_dst_connections]
            node_features = self.scaler.fit_transform(node_features)
            edge_list = [[ip_to_idx[row['_source_source_ip']], ip_to_idx[row['_source_destination_ip']]] for _, row in window_df.iterrows()]
            edge_index = np.array(edge_list).T
            graph = Data(x=node_features, edge_index=edge_index)
            label = int((window_df['label'] == 'malicious').any())
            X.append(graph)
            y.append(label)

        X_seq, y_seq = [], []
        for k in range(self.m, len(window_ids)):
            seq = X[k - self.m:k]
            sequence_label = int(any(y[k - self.m:k]))
            X_seq.append(seq)
            y_seq.append(sequence_label)

        n = len(X_seq)
        train_end = int(0.8 * n)
        val_end = int(0.9 * n)
        return (X_seq[:train_end], y_seq[:train_end], 
                X_seq[train_end:val_end], y_seq[train_end:val_end], 
                X_seq[val_end:], y_seq[val_end:])

import tensorflow as tf
from tensorflow.keras import layers

class GCNLayer(layers.Layer):
    def __init__(self, units, activation='relu'):
        super(GCNLayer, self).__init__()
        self.dense = layers.Dense(units, activation=None, use_bias=False,
                                 kernel_regularizer=tf.keras.regularizers.l2(1e-3))
        self.activation = tf.keras.activations.get(activation)
        self.batch_norm = layers.BatchNormalization()

    def call(self, node_features, adj_norm, training=False):
        h = tf.sparse.sparse_dense_matmul(adj_norm, node_features)
        h = self.dense(h)
        h = self.batch_norm(h, training=training)
        h = self.activation(h)
        return h

class GNN(tf.keras.Model):
    def __init__(self, hidden_units=128, output_units=64):
        super(GNN, self).__init__()
        self.gcn1 = GCNLayer(hidden_units)
        self.dropout1 = layers.Dropout(0.3)
        self.gcn2 = GCNLayer(output_units)
        self.dropout2 = layers.Dropout(0.3)

    def call(self, inputs, training=False):
        node_features, edge_indices, num_nodes = inputs
        adj_norm = compute_normalized_adjacency(edge_indices, num_nodes)
        x = self.gcn1(node_features, adj_norm, training=training)
        x = self.dropout1(x, training=training)
        x = self.gcn2(x, adj_norm, training=training)
        x = self.dropout2(x, training=training)
        embedding = tf.reduce_mean(x, axis=0)
        return embedding

def compute_normalized_adjacency(edge_indices, num_nodes):
    num_nodes = tf.cast(num_nodes, tf.int64)
    adj = tf.sparse.SparseTensor(indices=tf.transpose(edge_indices),
                                 values=tf.ones([tf.shape(edge_indices)[1]], dtype=tf.float32),
                                 dense_shape=[num_nodes, num_nodes])
    identity_indices = tf.stack([tf.range(num_nodes), tf.range(num_nodes)], axis=1)
    identity_sparse = tf.sparse.SparseTensor(indices=identity_indices,
                                            values=tf.ones([num_nodes], dtype=tf.float32),
                                            dense_shape=[num_nodes, num_nodes])
    adj = tf.sparse.add(adj, identity_sparse)
    degree = tf.sparse.reduce_sum(adj, axis=1)
    degree_inv_sqrt = tf.pow(degree + 1e-9, -0.5)
    degree_inv_sqrt = tf.where(tf.math.is_inf(degree_inv_sqrt), 0.0, degree_inv_sqrt)
    adj_norm = tf.sparse.SparseTensor(indices=adj.indices,
                                      values=adj.values * tf.gather(degree_inv_sqrt, adj.indices[:, 0]) * tf.gather(degree_inv_sqrt, adj.indices[:, 1]),
                                      dense_shape=adj.dense_shape)
    return adj_norm

class LSTMModel(tf.keras.Model):
    def __init__(self, input_size=64, hidden_size=128):
        super(LSTMModel, self).__init__()
        self.lstm = layers.LSTM(hidden_size, return_sequences=False,
                               kernel_regularizer=tf.keras.regularizers.l2(1e-3),
                               recurrent_regularizer=tf.keras.regularizers.l2(1e-3))
        self.dense = layers.Dense(1, activation='sigmoid',
                                 kernel_regularizer=tf.keras.regularizers.l2(1e-3))
        self.batch_norm = layers.BatchNormalization()

    def call(self, inputs, training=False):
        x = self.lstm(inputs)
        x = self.batch_norm(x, training=training)
        x = self.dense(x)
        return x

class RLTrainer:
    def __init__(self, X_train, y_train, X_val, y_val, model_path, m=3, num_episodes=100, batch_size=32, 
                 hidden_units=128, output_units=64, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.model_path = model_path
        self.m = m
        self.num_episodes = num_episodes
        self.batch_size = batch_size
        self.gnn = GNN(hidden_units=hidden_units, output_units=output_units)
        self.lstm = LSTMModel(input_size=output_units, hidden_size=128)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
        self.best_val_loss = float('inf')
        self.counter = 0
        self.patience = 10
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.memory = deque(maxlen=2000)
        self.target_gnn = GNN(hidden_units=hidden_units, output_units=output_units)
        self.target_lstm = LSTMModel(input_size=output_units, hidden_size=128)
        self.update_target_model()

    def update_target_model(self):
        self.target_gnn.set_weights(self.gnn.get_weights())
        self.target_lstm.set_weights(self.lstm.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(2)
        embeddings = self.get_embeddings(state)
        q_values = self.lstm(embeddings)
        return 0 if q_values.numpy()[0][0] < 0.5 else 1

    def get_embeddings(self, sequence):
        embeddings = []
        for graph in sequence:
            node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
            edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
            num_nodes = node_features.shape[0]
            embedding = self.gnn((node_features, edge_indices, num_nodes))
            embeddings.append(embedding)
        return tf.stack(embeddings)[None, :]

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory, self.batch_size)
        states = [m[0] for m in minibatch]
        actions = [m[1] for m in minibatch]
        rewards = [m[2] for m in minibatch]
        next_states = [m[3] for m in minibatch]
        dones = [m[4] for m in minibatch]

        state_embeddings = tf.stack([self.get_embeddings(s)[0] for s in states])
        next_state_embeddings = tf.stack([self.get_embeddings(ns)[0] for ns in next_states])
        
        with tf.GradientTape() as tape:
            q_values = self.lstm(state_embeddings)
            target_q_values = self.target_lstm(next_state_embeddings)
            targets = []
            for i in range(self.batch_size):
                target = rewards[i] if dones[i] else rewards[i] + self.gamma * tf.reduce_max(target_q_values[i])
                targets.append(target)
            targets = tf.convert_to_tensor(targets, dtype=tf.float32)[:, None]
            loss = tf.keras.losses.mean_squared_error(targets, q_values)
        
        grads = tape.gradient(loss, self.gnn.trainable_variables + self.lstm.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.gnn.trainable_variables + self.lstm.trainable_variables))
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def train(self):
        for episode in range(self.num_episodes):
            train_data = list(zip(self.X_train, self.y_train))
            random.shuffle(train_data)
            total_reward = 0
            for i, (sequence, label) in enumerate(train_data):
                action = self.act(sequence)
                reward = 1 if action == label else -1
                total_reward += reward
                next_idx = min(i + 1, len(train_data) - 1)
                next_sequence, _ = train_data[next_idx]
                done = i == len(train_data) - 1
                self.remember(sequence, action, reward, next_sequence, done)
                self.replay()

            val_loss = 0
            for sequence, label in zip(self.X_val, self.y_val):
                embeddings = self.get_embeddings(sequence)
                q_values = self.lstm(embeddings)
                pred = 0 if q_values.numpy()[0][0] < 0.5 else 1
                val_loss += tf.keras.losses.binary_crossentropy([label], [pred]).numpy()
            val_loss /= len(self.X_val)
            print(f"Episode {episode}, Reward: {total_reward}, Val Loss: {val_loss:.4f}, Epsilon: {self.epsilon:.4f}")

            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.counter = 0
                self.gnn.save_weights(f"{self.model_path}_gnn.h5")
                self.lstm.save_weights(f"{self.model_path}_lstm.h5")
                self.update_target_model()
            else:
                self.counter += 1
                if self.counter >= self.patience:
                    print("Early stopping triggered")
                    break

        self.gnn.load_weights(f"{self.model_path}_gnn.h5")
        self.lstm.load_weights(f"{self.model_path}_lstm.h5")

class ModelEvaluator:
    def __init__(self, gnn, lstm):
        self.gnn = gnn
        self.lstm = lstm

    def evaluate(self, X_val, y_val):
        val_loss = 0
        correct = 0
        total = 0
        for sequence, label in zip(X_val, y_val):
            embeddings = []
            for graph in sequence:
                node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                num_nodes = node_features.shape[0]
                embedding = self.gnn((node_features, edge_indices, num_nodes))
                embeddings.append(embedding)
            sequence_embeddings = tf.stack(embeddings)[None, :]
            y_pred = self.lstm(sequence_embeddings).numpy()[0][0]
            val_loss += tf.keras.losses.binary_crossentropy([label], [y_pred]).numpy()
            if (y_pred > 0.5) == label:
                correct += 1
            total += 1
        val_loss /= total
        val_accuracy = correct / total
        return val_loss, val_accuracy

class ModelTester:
    def __init__(self, model):
        self.gnn, self.lstm = model
        self.scaler = StandardScaler()

    def test(self, X_test, y_test):
        correct = 0
        total = 0
        for sequence, label in zip(X_test, y_test):
            embeddings = []
            for graph in sequence:
                node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                num_nodes = node_features.shape[0]
                embedding = self.gnn((node_features, edge_indices, num_nodes))
                embeddings.append(embedding)
            sequence_embeddings = tf.stack(embeddings)[None, :]
            y_pred = self.lstm(sequence_embeddings).numpy()[0][0]
            if (y_pred > 0.5) == label:
                correct += 1
            total += 1
        test_accuracy = correct / total
        print(f"Test Accuracy: {test_accuracy:.4f}")
        return test_accuracy

if __name__ == '__main__':
    dataset_path = 'C:\Users\ASUS\Guidewire_Hackathon\datasets\elastic_may2021_malicious_data.csv'
    test_dataset_path = 'C:\Users\ASUS\Guidewire_Hackathon\datasets\elastic_may2022_data.csv'
    model_path = 'C:\Users\ASUS\Guidewire_Hackathon\src\models\trained_hybrid_model1'
    preprocessor = DataPreprocessor(dataset_path, m=3)
    X_train, y_train, X_val, y_val, X_test, y_test = preprocessor.load_and_preprocess_data()
    trainer = RLTrainer(X_train, y_train, X_val, y_val, model_path, m=3, num_episodes=50, batch_size=32, 
                         hidden_units=128, output_units=64)
    trainer.train()
    evaluator = ModelEvaluator(trainer.gnn, trainer.lstm)
    val_loss, val_accuracy = evaluator.evaluate(X_val, y_val)
    tester = ModelTester((trainer.gnn, trainer.lstm))
    test_accuracy = tester.test(X_test, y_test)
    new_test_accuracy = tester.preprocess_and_test(test_dataset_path, m=3)
    print(f"New Test Accuracy: {new_test_accuracy:.4f}")


In [ ]:
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
import random
from collections import deque
import tensorflow as tf
from tensorflow.keras import layers  # type: ignore

# ---------------- Data Preprocessing Class ----------------
class DataPreprocessor:
    def __init__(self, dataset_path, m=3):
        self.dataset_path = dataset_path
        self.m = m
        self.scaler = StandardScaler()

    def load_and_preprocess_data(self):
        # Load selected columns from CSV file
        df = pd.read_csv(self.dataset_path, usecols=['_source_source_ip', '_source_destination_ip', 
                                                      '_source_network_bytes', '_source_@timestamp', 'label'])
        # Convert timestamp column to datetime
        df['timestamp'] = pd.to_datetime(df['_source_@timestamp'])
        # Create a window ID based on minute-level granularity
        df['window_id'] = df['timestamp'].dt.floor('T').astype('int64') // 10**9 // 60
        grouped = df.groupby('window_id')
        window_ids = np.array(sorted(grouped.groups.keys()))

        X, y = [], []
        # Process each time window to construct graph data
        for wid in window_ids:
            window_df = grouped.get_group(wid)
            all_ips = np.unique(np.concatenate([window_df['_source_source_ip'].unique(), window_df['_source_destination_ip'].unique()]))
            ip_to_idx = {ip: i for i, ip in enumerate(all_ips)}
            num_nodes = len(all_ips)
            node_features = np.zeros((num_nodes, 4))
            for ip in all_ips:
                src_bytes = window_df[window_df['_source_source_ip'] == ip]['_source_network_bytes'].sum()
                dst_bytes = window_df[window_df['_source_destination_ip'] == ip]['_source_network_bytes'].sum()
                num_src_connections = len(window_df[window_df['_source_source_ip'] == ip])
                num_dst_connections = len(window_df[window_df['_source_destination_ip'] == ip])
                node_features[ip_to_idx[ip]] = [src_bytes, dst_bytes, num_src_connections, num_dst_connections]
            node_features = self.scaler.fit_transform(node_features)
            edge_list = [[ip_to_idx[row['_source_source_ip']], ip_to_idx[row['_source_destination_ip']]] for _, row in window_df.iterrows()]
            edge_index = np.array(edge_list).T
            graph = Data(x=node_features, edge_index=edge_index)
            label = int((window_df['label'] == 'malicious').any())
            X.append(graph)
            y.append(label)

        # Create sequences of graphs using sliding window of size m
        X_seq, y_seq = [], []
        for k in range(self.m, len(window_ids)):
            seq = X[k - self.m:k]
            sequence_label = int(any(y[k - self.m:k]))
            X_seq.append(seq)
            y_seq.append(sequence_label)

        n = len(X_seq)
        train_end = int(0.8 * n)
        val_end = int(0.9 * n)
        return (X_seq[:train_end], y_seq[:train_end], 
                X_seq[train_end:val_end], y_seq[train_end:val_end], 
                X_seq[val_end:], y_seq[val_end:])

# ---------------- GCN Layer and GNN Model ----------------
class GCNLayer(layers.Layer):
    def __init__(self, units, activation='relu'):
        super(GCNLayer, self).__init__()
        self.dense = layers.Dense(units, activation=None, use_bias=False,
                                  kernel_regularizer=tf.keras.regularizers.l2(1e-3))
        self.activation = tf.keras.activations.get(activation)
        self.batch_norm = layers.BatchNormalization()

    def call(self, node_features, adj_norm, training=False):
        # Multiply adjacency matrix with node features, then apply dense layer, batch norm, and activation
        h = tf.sparse.sparse_dense_matmul(adj_norm, node_features)
        h = self.dense(h)
        h = self.batch_norm(h, training=training)
        h = self.activation(h)
        return h

class GNN(tf.keras.Model):
    def __init__(self, hidden_units=128, output_units=64):
        super(GNN, self).__init__()
        self.gcn1 = GCNLayer(hidden_units)
        self.dropout1 = layers.Dropout(0.3)
        self.gcn2 = GCNLayer(output_units)
        self.dropout2 = layers.Dropout(0.3)

    def call(self, inputs, training=False):
        node_features, edge_indices, num_nodes = inputs
        adj_norm = compute_normalized_adjacency(edge_indices, num_nodes)
        x = self.gcn1(node_features, adj_norm, training=training)
        x = self.dropout1(x, training=training)
        x = self.gcn2(x, adj_norm, training=training)
        x = self.dropout2(x, training=training)
        # Aggregate node embeddings to get graph-level representation
        embedding = tf.reduce_mean(x, axis=0)
        return embedding

def compute_normalized_adjacency(edge_indices, num_nodes):
    # Compute normalized adjacency matrix for GCN
    num_nodes = tf.cast(num_nodes, tf.int64)
    adj = tf.sparse.SparseTensor(indices=tf.transpose(edge_indices),
                                 values=tf.ones([tf.shape(edge_indices)[1]], dtype=tf.float32),
                                 dense_shape=[num_nodes, num_nodes])
    identity_indices = tf.stack([tf.range(num_nodes), tf.range(num_nodes)], axis=1)
    identity_sparse = tf.sparse.SparseTensor(indices=identity_indices,
                                             values=tf.ones([num_nodes], dtype=tf.float32),
                                             dense_shape=[num_nodes, num_nodes])
    adj = tf.sparse.add(adj, identity_sparse)
    degree = tf.sparse.reduce_sum(adj, axis=1)
    degree_inv_sqrt = tf.pow(degree + 1e-9, -0.5)
    degree_inv_sqrt = tf.where(tf.math.is_inf(degree_inv_sqrt), 0.0, degree_inv_sqrt)
    adj_norm = tf.sparse.SparseTensor(indices=adj.indices,
                                      values=adj.values * tf.gather(degree_inv_sqrt, adj.indices[:, 0]) * tf.gather(degree_inv_sqrt, adj.indices[:, 1]),
                                      dense_shape=adj.dense_shape)
    return adj_norm

# ---------------- LSTM Model ----------------
class LSTMModel(tf.keras.Model):
    def __init__(self, input_size=64, hidden_size=128):
        super(LSTMModel, self).__init__()
        self.lstm = layers.LSTM(hidden_size, return_sequences=False,
                                kernel_regularizer=tf.keras.regularizers.l2(1e-3),
                                recurrent_regularizer=tf.keras.regularizers.l2(1e-3))
        self.dense = layers.Dense(1, activation='sigmoid',
                                  kernel_regularizer=tf.keras.regularizers.l2(1e-3))
        self.batch_norm = layers.BatchNormalization()

    def call(self, inputs, training=False):
        x = self.lstm(inputs)
        x = self.batch_norm(x, training=training)
        x = self.dense(x)
        return x

# ---------------- RL Trainer (Hybrid Model Trainer using RL) ----------------
class RLTrainer:
    def __init__(self, X_train, y_train, X_val, y_val, model_path, m=3, num_episodes=50, batch_size=32, 
                 hidden_units=128, output_units=64, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.995):
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.model_path = model_path
        self.m = m
        self.num_episodes = num_episodes
        self.batch_size = batch_size
        self.gnn = GNN(hidden_units=hidden_units, output_units=output_units)
        self.lstm = LSTMModel(input_size=output_units, hidden_size=128)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
        self.best_val_loss = float('inf')
        self.counter = 0
        self.patience = 10
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.memory = deque(maxlen=2000)
        self.target_gnn = GNN(hidden_units=hidden_units, output_units=output_units)
        self.target_lstm = LSTMModel(input_size=output_units, hidden_size=128)
        self.update_target_model()

    def update_target_model(self):
        # Update target model weights
        self.target_gnn.set_weights(self.gnn.get_weights())
        self.target_lstm.set_weights(self.lstm.get_weights())

    def remember(self, state, action, reward, next_state, done):
        # Store experience in replay memory
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        # Epsilon-greedy action selection
        if np.random.rand() <= self.epsilon:
            return random.randrange(2)
        embeddings = self.get_embeddings(state)
        q_values = self.lstm(embeddings)
        return 0 if q_values.numpy()[0][0] < 0.5 else 1

    def get_embeddings(self, sequence):
        # Compute graph embeddings for each graph in the sequence using the GNN
        embeddings = []
        for graph in sequence:
            node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
            edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
            num_nodes = node_features.shape[0]
            embedding = self.gnn((node_features, edge_indices, num_nodes))
            embeddings.append(embedding)
        return tf.stack(embeddings)[None, :]

    def replay(self):
        # Perform experience replay for training
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory, self.batch_size)
        states = [m[0] for m in minibatch]
        actions = [m[1] for m in minibatch]
        rewards = [m[2] for m in minibatch]
        next_states = [m[3] for m in minibatch]
        dones = [m[4] for m in minibatch]

        state_embeddings = tf.stack([self.get_embeddings(s)[0] for s in states])
        next_state_embeddings = tf.stack([self.get_embeddings(ns)[0] for ns in next_states])
        
        with tf.GradientTape() as tape:
            q_values = self.lstm(state_embeddings)
            target_q_values = self.target_lstm(next_state_embeddings)
            targets = []
            for i in range(self.batch_size):
                target = rewards[i] if dones[i] else rewards[i] + self.gamma * tf.reduce_max(target_q_values[i])
                targets.append(target)
            targets = tf.convert_to_tensor(targets, dtype=tf.float32)[:, None]
            loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(targets, q_values))
        grads = tape.gradient(loss, self.gnn.trainable_variables + self.lstm.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.gnn.trainable_variables + self.lstm.trainable_variables))
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def train(self):
        # Training loop over episodes
        for episode in range(self.num_episodes):
            train_data = list(zip(self.X_train, self.y_train))
            random.shuffle(train_data)
            total_reward = 0
            for i, (sequence, label) in enumerate(train_data):
                action = self.act(sequence)
                reward = 1 if action == label else -1
                total_reward += reward
                next_idx = min(i + 1, len(train_data) - 1)
                next_sequence, _ = train_data[next_idx]
                done = i == len(train_data) - 1
                self.remember(sequence, action, reward, next_sequence, done)
                self.replay()

            val_loss = 0
            for sequence, label in zip(self.X_val, self.y_val):
                embeddings = self.get_embeddings(sequence)
                q_values = self.lstm(embeddings)
                pred = 0 if q_values.numpy()[0][0] < 0.5 else 1
                val_loss += tf.keras.losses.binary_crossentropy([label], [pred]).numpy()
            val_loss /= len(self.X_val)
            print(f"Episode {episode}, Reward: {total_reward}, Val Loss: {val_loss:.4f}, Epsilon: {self.epsilon:.4f}")

            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.counter = 0
                self.gnn.save_weights(f"{self.model_path}_gnn.h5")
                self.lstm.save_weights(f"{self.model_path}_lstm.h5")
                self.update_target_model()
            else:
                self.counter += 1
                if self.counter >= self.patience:
                    print("Early stopping triggered")
                    break

        self.gnn.load_weights(f"{self.model_path}_gnn.h5")
        self.lstm.load_weights(f"{self.model_path}_lstm.h5")

# ---------------- Model Evaluator ----------------
class ModelEvaluator:
    def __init__(self, gnn, lstm):
        self.gnn = gnn
        self.lstm = lstm

    def evaluate(self, X_val, y_val):
        # Evaluate model on validation set
        val_loss = 0
        correct = 0
        total = 0
        for sequence, label in zip(X_val, y_val):
            embeddings = []
            for graph in sequence:
                node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                num_nodes = node_features.shape[0]
                embedding = self.gnn((node_features, edge_indices, num_nodes))
                embeddings.append(embedding)
            sequence_embeddings = tf.stack(embeddings)[None, :]
            y_pred = self.lstm(sequence_embeddings).numpy()[0][0]
            val_loss += tf.keras.losses.binary_crossentropy([label], [y_pred]).numpy()
            if (y_pred > 0.5) == label:
                correct += 1
            total += 1
        val_loss /= total
        val_accuracy = correct / total
        return val_loss, val_accuracy

# ---------------- Model Tester ----------------
class ModelTester:
    def __init__(self, model):
        self.gnn, self.lstm = model
        self.scaler = StandardScaler()

    def test(self, X_test, y_test):
        # Evaluate model on test set and print accuracy
        correct = 0
        total = 0
        for sequence, label in zip(X_test, y_test):
            embeddings = []
            for graph in sequence:
                node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                num_nodes = node_features.shape[0]
                embedding = self.gnn((node_features, edge_indices, num_nodes))
                embeddings.append(embedding)
            sequence_embeddings = tf.stack(embeddings)[None, :]
            y_pred = self.lstm(sequence_embeddings).numpy()[0][0]
            if (y_pred > 0.5) == label:
                correct += 1
            total += 1
        test_accuracy = correct / total
        print(f"Test Accuracy: {test_accuracy:.4f}")
        return test_accuracy

    def preprocess_and_test(self, test_dataset_path, m=3):
        # Load test dataset, preprocess and create dummy graph sequences (placeholder)
        df = pd.read_csv(test_dataset_path)
        X_test = []
        y_test = []
        for i in range(0, len(df) - m, m):
            sequence = []
            for j in range(m):
                graph = {
                    'x': df.iloc[i+j][['feature1', 'feature2', 'feature3']].values.astype(np.float32),
                    'edge_index': np.array([[0, 1], [1, 0]], dtype=np.int64)
                }
                sequence.append(graph)
            X_test.append(sequence)
            y_test.append(int(df.iloc[i]['label'] == 'malicious'))
        return self.test(X_test, y_test)

if __name__ == '__main__':
    dataset_path = 'C:\\Users\\ASUS\\Guidewire_Hackathon\\datasets\\elastic_may2021_malicious_data.csv'
    test_dataset_path = 'C:\\Users\\ASUS\\Guidewire_Hackathon\\datasets\\elastic_may2022_data.csv'
    model_path = 'C:\\Users\\ASUS\\Guidewire_Hackathon\\src\\models\\trained_hybrid_model1'
    preprocessor = DataPreprocessor(dataset_path, m=3)
    X_train, y_train, X_val, y_val, X_test, y_test = preprocessor.load_and_preprocess_data()
    trainer = RLTrainer(X_train, y_train, X_val, y_val, model_path, m=3, num_episodes=50, batch_size=32, 
                         hidden_units=128, output_units=64)
    trainer.train()
    evaluator = ModelEvaluator(trainer.gnn, trainer.lstm)
    val_loss, val_accuracy = evaluator.evaluate(X_val, y_val)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    tester = ModelTester((trainer.gnn, trainer.lstm))
    test_accuracy = tester.test(X_test, y_test)
    new_test_accuracy = tester.preprocess_and_test(test_dataset_path, m=3)
    print(f"New Test Accuracy: {new_test_accuracy:.4f}")


In [3]:
# data_preprocessing.py
import pandas as pd
import numpy as np
from torch_geometric.data import Data
from sklearn.model_selection import train_test_split

class DataPreprocessor:
    def __init__(self, dataset_path, m=3):
        self.dataset_path = dataset_path
        self.m = m

    def load_and_preprocess_data(self):
        df = pd.read_csv(self.dataset_path, usecols=['_source_source_ip', '_source_destination_ip', 
                                                     '_source_network_bytes', '_source_@timestamp', 'label'])
        df['timestamp'] = pd.to_datetime(df['_source_@timestamp'])
        df['window_id'] = df['timestamp'].dt.floor('T').astype('int64') // 10**9 // 60
        grouped = df.groupby('window_id')
        window_ids = np.array(sorted(grouped.groups.keys()))

        X, y = [], []
        for wid in window_ids:
            window_df = grouped.get_group(wid)
            all_ips = np.unique(np.concatenate([window_df['_source_source_ip'], window_df['_source_destination_ip']]))
            ip_to_idx = {ip: i for i, ip in enumerate(all_ips)}
            num_nodes = len(all_ips)
            node_features = np.zeros((num_nodes, 4))  # [src_bytes, dst_bytes, src_conn, dst_conn]
            for _, row in window_df.iterrows():
                src_idx = ip_to_idx[row['_source_source_ip']]
                dst_idx = ip_to_idx[row['_source_destination_ip']]
                node_features[src_idx, 0] += row['_source_network_bytes']  # src_bytes
                node_features[dst_idx, 1] += row['_source_network_bytes']  # dst_bytes
                node_features[src_idx, 2] += 1  # src_conn
                node_features[dst_idx, 3] += 1  # dst_conn
            edge_index = np.array([[ip_to_idx[row['_source_source_ip']], ip_to_idx[row['_source_destination_ip']]] 
                                   for _, row in window_df.iterrows()]).T
            graph = Data(x=node_features, edge_index=edge_index)
            label = int((window_df['label'] == 'malicious').any())
            X.append(graph)
            y.append(label)

        X_seq, y_seq = [], []
        for k in range(self.m, len(window_ids)):
            seq = X[k - self.m:k]
            sequence_label = int(any(y[k - self.m:k]))
            X_seq.append(seq)
            y_seq.append(sequence_label)

        X_temp, X_test, y_temp, y_test = train_test_split(X_seq, y_seq, test_size=0.1, stratify=y_seq, random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1111, stratify=y_temp, random_state=42)

        return X_train, y_train, X_val, y_val, X_test, y_test

In [4]:
# GNN.py
import tensorflow as tf

class GCNLayer(tf.keras.layers.Layer):
    def __init__(self, units, activation='relu'):
        super(GCNLayer, self).__init__()
        self.dense = tf.keras.layers.Dense(units, activation=None, use_bias=False,
                                          kernel_regularizer=tf.keras.regularizers.l2(1e-2))
        self.activation = tf.keras.activations.get(activation)

    def call(self, node_features, adj_norm):
        h = tf.sparse.sparse_dense_matmul(adj_norm, node_features)
        h = self.dense(h)
        h = self.activation(h)
        return h

class GNN(tf.keras.Model):
    def __init__(self, hidden_units=64, output_units=16):
        super(GNN, self).__init__()
        self.gcn1 = GCNLayer(hidden_units)
        self.dropout1 = tf.keras.layers.Dropout(0.3)
        self.gcn2 = GCNLayer(output_units)
        self.dropout2 = tf.keras.layers.Dropout(0.3)

    def call(self, inputs, training=False):
        node_features, edge_indices, num_nodes = inputs
        adj_norm = compute_normalized_adjacency(edge_indices, num_nodes)
        x = self.gcn1(node_features, adj_norm)
        x = self.dropout1(x, training=training)
        x = self.gcn2(x, adj_norm)
        x = self.dropout2(x, training=training)
        embedding = tf.reduce_mean(x, axis=0)
        return embedding

def compute_normalized_adjacency(edge_indices, num_nodes):
    num_nodes = tf.cast(num_nodes, tf.int64)
    adj = tf.sparse.SparseTensor(indices=tf.transpose(edge_indices),
                                 values=tf.ones([tf.shape(edge_indices)[1]], dtype=tf.float32),
                                 dense_shape=[num_nodes, num_nodes])
    identity_indices = tf.stack([tf.range(num_nodes), tf.range(num_nodes)], axis=1)
    identity_sparse = tf.sparse.SparseTensor(indices=identity_indices,
                                            values=tf.ones([num_nodes], dtype=tf.float32),
                                            dense_shape=[num_nodes, num_nodes])
    adj = tf.sparse.add(adj, identity_sparse)
    degree = tf.sparse.reduce_sum(adj, axis=1)
    degree_inv_sqrt = tf.pow(degree + 1e-9, -0.5)
    degree_inv_sqrt = tf.where(tf.math.is_inf(degree_inv_sqrt), 0.0, degree_inv_sqrt)
    adj_norm = tf.sparse.SparseTensor(indices=adj.indices,
                                      values=adj.values * tf.gather(degree_inv_sqrt, adj.indices[:, 0]) * tf.gather(degree_inv_sqrt, adj.indices[:, 1]),
                                      dense_shape=adj.dense_shape)
    return adj_norm

In [6]:
# LSTM.py
import tensorflow as tf

class LSTMModel(tf.keras.Model):
    def __init__(self, input_size=16, hidden_size=32):
        super(LSTMModel, self).__init__()
        self.lstm = tf.keras.layers.LSTM(hidden_size, return_sequences=False,
                                        kernel_regularizer=tf.keras.regularizers.l2(1e-2),
                                        recurrent_regularizer=tf.keras.regularizers.l2(1e-2))
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid',
                                          kernel_regularizer=tf.keras.regularizers.l2(1e-2))

    def call(self, inputs, training=False):
        x = self.lstm(inputs)
        x = self.dense(x)
        return x

In [7]:
# rl_agent.py
import tensorflow as tf
import numpy as np
from collections import deque
import random

class DQNAgent:
    def __init__(self, state_size=16, action_size=2):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()

    def _build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, input_dim=self.state_size, activation='relu'),
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state[np.newaxis, :])
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        states = np.array([t[0] for t in minibatch])
        actions = np.array([t[1] for t in minibatch])
        rewards = np.array([t[2] for t in minibatch])
        next_states = np.array([t[3] for t in minibatch])
        dones = np.array([t[4] for t in minibatch])

        targets = self.model.predict(states)
        targets_next = self.target_model.predict(next_states)

        for i in range(batch_size):
            targets[i][actions[i]] = rewards[i] + self.gamma * np.amax(targets_next[i]) * (1 - dones[i])

        self.model.fit(states, targets, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [8]:
# training.py
import tensorflow as tf
import random
import matplotlib.pyplot as plt


class ModelTrainer:
    def __init__(self, X_train, y_train, X_val, y_val, model_path, m=3, num_episodes=100, batch_size=32, hidden_units=64, output_units=16):
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.model_path = model_path
        self.m = m
        self.num_episodes = num_episodes
        self.batch_size = batch_size
        self.gnn = GNN(hidden_units=hidden_units, output_units=output_units)
        self.lstm = LSTMModel(input_size=output_units, hidden_size=32)
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.rl_agent = DQNAgent(state_size=output_units, action_size=2)
        self.best_val_loss = float('inf')
        self.counter = 0
        self.patience = 10
        self.train_losses = []
        self.val_losses = []

    def train(self):
        for episode in range(self.num_episodes):
            train_data = list(zip(self.X_train, self.y_train))
            random.shuffle(train_data)
            train_loss = 0
            train_correct = 0
            train_total = 0

            for i in range(0, len(train_data), self.batch_size):
                batch = train_data[i:i + self.batch_size]
                batch_X = [seq for seq, _ in batch]
                batch_y = tf.convert_to_tensor([label for _, label in batch], dtype=tf.float32)[:, None]

                with tf.GradientTape() as tape:
                    batch_embeddings = []
                    for sequence in batch_X:
                        sequence_embeddings = []
                        for graph in sequence:
                            node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                            edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                            num_nodes = node_features.shape[0]
                            embedding = self.gnn((node_features, edge_indices, num_nodes), training=True)
                            sequence_embeddings.append(embedding)
                        batch_embeddings.append(tf.stack(sequence_embeddings))
                    batch_embeddings = tf.stack(batch_embeddings)
                    y_pred = self.lstm(batch_embeddings, training=True)
                    
                    # RL integration
                    state = tf.reduce_mean(batch_embeddings, axis=1).numpy()
                    action = self.rl_agent.act(state[0])
                    reward = -tf.keras.losses.binary_crossentropy(batch_y, y_pred).numpy().mean()
                    next_state = tf.reduce_mean(batch_embeddings, axis=1).numpy()
                    done = (episode == self.num_episodes - 1)
                    self.rl_agent.remember(state[0], action, reward, next_state[0], done)
                    
                    if len(self.rl_agent.memory) > self.batch_size:
                        self.rl_agent.replay(self.batch_size)

                    class_weights = {0: 1.0, 1: 5.0}
                    weights = tf.gather(tf.constant([class_weights[0], class_weights[1]]), tf.cast(batch_y, tf.int32))
                    loss = tf.keras.losses.binary_crossentropy(batch_y, y_pred) * weights
                    loss = tf.reduce_mean(loss)

                grads = tape.gradient(loss, self.gnn.trainable_variables + self.lstm.trainable_variables)
                self.optimizer.apply_gradients(zip(grads, self.gnn.trainable_variables + self.lstm.trainable_variables))
                train_loss += loss.numpy()
                train_correct += tf.reduce_sum(tf.cast(tf.equal(tf.cast(y_pred > 0.5, tf.float32), batch_y), tf.int32)).numpy()
                train_total += len(batch_y)

            train_loss /= (len(train_data) + self.batch_size - 1) // self.batch_size
            train_acc = train_correct / train_total

            val_data = list(zip(self.X_val, self.y_val))
            val_loss = 0
            val_correct = 0
            val_total = 0
            for i in range(0, len(val_data), self.batch_size):
                batch = val_data[i:i + self.batch_size]
                batch_X = [seq for seq, _ in batch]
                batch_y = tf.convert_to_tensor([label for _, label in batch], dtype=tf.float32)[:, None]
                batch_embeddings = []
                for sequence in batch_X:
                    sequence_embeddings = []
                    for graph in sequence:
                        node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                        edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                        num_nodes = node_features.shape[0]
                        embedding = self.gnn((node_features, edge_indices, num_nodes), training=False)
                        sequence_embeddings.append(embedding)
                    batch_embeddings.append(tf.stack(sequence_embeddings))
                batch_embeddings = tf.stack(batch_embeddings)
                y_pred = self.lstm(batch_embeddings, training=False)
                val_loss += tf.keras.losses.binary_crossentropy(batch_y, y_pred).numpy().mean()
                val_correct += tf.reduce_sum(tf.cast(tf.equal(tf.cast(y_pred > 0.5, tf.float32), batch_y), tf.int32)).numpy()
                val_total += len(batch_y)

            val_loss /= (len(val_data) + self.batch_size - 1) // self.batch_size
            val_acc = val_correct / val_total

            self.train_losses.append(train_loss)
            self.val_losses.append(val_loss)
            print(f"Episode {episode}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.counter = 0
                self.gnn.save_weights(f"{self.model_path}_gnn.h5")
                self.lstm.save_weights(f"{self.model_path}_lstm.h5")
                self.rl_agent.save(f"{self.model_path}_dqn.h5")
            else:
                self.counter += 1
                if self.counter >= self.patience:
                    print("Early stopping triggered")
                    break

            if episode % 10 == 0:
                self.rl_agent.update_target_model()

        self.gnn.load_weights(f"{self.model_path}_gnn.h5")
        self.lstm.load_weights(f"{self.model_path}_lstm.h5")
        self.rl_agent.load(f"{self.model_path}_dqn.h5")

        plt.plot(self.train_losses, label='Train Loss')
        plt.plot(self.val_losses, label='Val Loss')
        plt.xlabel('Episode')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

In [9]:
# evaluation.py
import tensorflow as tf

class ModelEvaluator:
    def __init__(self, gnn, lstm):
        self.gnn = gnn
        self.lstm = lstm

    def evaluate(self, X_val, y_val):
        val_loss = 0
        correct = 0
        total = 0
        for sequence, label in zip(X_val, y_val):
            embeddings = []
            for graph in sequence:
                node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                num_nodes = node_features.shape[0]
                embedding = self.gnn((node_features, edge_indices, num_nodes), training=False)
                embeddings.append(embedding)
            sequence_embeddings = tf.stack(embeddings)[None, :]
            y_pred = self.lstm(sequence_embeddings, training=False).numpy()[0][0]
            val_loss += tf.keras.losses.binary_crossentropy([label], [y_pred]).numpy()
            if (y_pred > 0.5) == label:
                correct += 1
            total += 1
        val_loss /= total
        val_accuracy = correct / total
        return val_loss, val_accuracy

In [11]:
# testing.py
import tensorflow as tf
import pandas as pd
import numpy as np
from torch_geometric.data import Data

class ModelTester:
    def __init__(self, model):
        self.gnn, self.lstm = model

    def test(self, X_test, y_test):
        correct = 0
        total = 0
        for sequence, label in zip(X_test, y_test):
            embeddings = []
            for graph in sequence:
                node_features = tf.convert_to_tensor(graph['x'], dtype=tf.float32)
                edge_indices = tf.convert_to_tensor(graph['edge_index'], dtype=tf.int64)
                num_nodes = node_features.shape[0]
                embedding = self.gnn((node_features, edge_indices, num_nodes), training=False)
                embeddings.append(embedding)
            sequence_embeddings = tf.stack(embeddings)[None, :]
            y_pred = self.lstm(sequence_embeddings, training=False).numpy()[0][0]
            if (y_pred > 0.5) == label:
                correct += 1
            total += 1
        test_accuracy = correct / total
        print(f"Test Accuracy: {test_accuracy:.4f}")
        return test_accuracy

    def preprocess_and_test(self, dataset_path, m=3):
        df = pd.read_csv(dataset_path, usecols=['_source_source_ip', '_source_destination_ip', 
                                                '_source_network_bytes', '_source_@timestamp', 'label'], delimiter=';')
        df['timestamp'] = pd.to_datetime(df['_source_@timestamp'])
        df['window_id'] = df['timestamp'].dt.floor('T').astype('int64') // 10**9 // 60
        grouped = df.groupby('window_id')
        window_ids = np.array(sorted(grouped.groups.keys()))

        X, y = [], []
        for wid in window_ids:
            window_df = grouped.get_group(wid)
            all_ips = np.unique(np.concatenate([window_df['_source_source_ip'], window_df['_source_destination_ip']]))
            ip_to_idx = {ip: i for i, ip in enumerate(all_ips)}
            num_nodes = len(all_ips)
            node_features = np.zeros((num_nodes, 4))
            for _, row in window_df.iterrows():
                src_idx = ip_to_idx[row['_source_source_ip']]
                dst_idx = ip_to_idx[row['_source_destination_ip']]
                node_features[src_idx, 0] += row['_source_network_bytes']
                node_features[dst_idx, 1] += row['_source_network_bytes']
                node_features[src_idx, 2] += 1
                node_features[dst_idx, 3] += 1
            edge_index = np.array([[ip_to_idx[row['_source_source_ip']], ip_to_idx[row['_source_destination_ip']]] 
                                   for _, row in window_df.iterrows()]).T
            graph = Data(x=node_features, edge_index=edge_index)
            label = int((window_df['label'] == 'malicious').any())
            X.append(graph)
            y.append(label)

        X_seq, y_seq = [], []
        for k in range(m, len(window_ids)):
            seq = X[k - m:k]
            sequence_label = int(any(y[k - m:k]))
            X_seq.append(seq)
            y_seq.append(sequence_label)

        print(f"Testing on preprocessed dataset from {dataset_path}")
        return self.test(X_seq, y_seq)

In [12]:
# main.py
import numpy as np
# from data_preprocessing import DataPreprocessor
# from training import ModelTrainer
# from evaluation import ModelEvaluator
# from testing import ModelTester

if __name__ == "__main__":
    dataset_path = "C:\\Users\\ASUS\\Guidewire_Hackathon\\datasets\\elastic_may2021_malicious_data.csv"
    test_dataset_path = "C:\\Users\\ASUS\\Guidewire_Hackathon\\datasets\\elastic_may2022_data.csv"
    model_path = "C:\\Users\\ASUS\\Guidewire_Hackathon\\src\\models\\trained_hybrid_model1"

    preprocessor = DataPreprocessor(dataset_path, m=3)
    X_train, y_train, X_val, y_val, X_test, y_test = preprocessor.load_and_preprocess_data()

    print("Training label distribution:", np.bincount(y_train))
    print("Validation label distribution:", np.bincount(y_val))
    print("Test label distribution:", np.bincount(y_test))

    trainer = ModelTrainer(X_train, y_train, X_val, y_val, model_path, m=3, num_episodes=100, batch_size=32, hidden_units=64, output_units=16)
    trainer.train()

    evaluator = ModelEvaluator(trainer.gnn, trainer.lstm)
    val_loss, val_accuracy = evaluator.evaluate(X_val, y_val)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    tester = ModelTester((trainer.gnn, trainer.lstm))
    test_accuracy = tester.test(X_test, y_test)
    new_test_accuracy = tester.preprocess_and_test(test_dataset_path, m=3)
    print(f"New Test Accuracy: {new_test_accuracy:.4f}")

Training label distribution: [   0 1192]
Validation label distribution: [  0 150]
Test label distribution: [  0 150]


LookupError: No gradient defined for operation'IteratorGetNext' (op type: IteratorGetNext). In general every operation must have an associated `@tf.RegisterGradient` for correct autodiff, which this op is lacking. If you want to pretend this operation is a constant in your program, you may insert `tf.stop_gradient`. This can be useful to silence the error in cases where you know gradients are not needed, e.g. the forward pass of tf.custom_gradient. Please see more details in https://www.tensorflow.org/api_docs/python/tf/custom_gradient.